# Welcome to pyHaiCS

In [1]:
import pyHaiCS as haics

In [2]:
print(f"Running pyHaiCS v.{haics.__version__}")

Running pyHaiCS v.0.0.1


## Example - Bayesian Logistic Regression + HMC for Breast Cancer Classification

In [10]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)